Initial data exploration & some preparation of it in order to start getting ready to feed some models :)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from pathlib import Path
import collections
import matplotlib.pyplot as plt

In [ ]:
input_path = Path('../input/stanford-covid-vaccine')
os.listdir(input_path)

In [ ]:
train_df = pd.read_json(input_path/'train.json', lines=True)
test_df = pd.read_json(input_path/'test.json', lines=True)
sample_sub = pd.read_csv(input_path/'sample_submission.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

So we are attempting to predict the degredation (& reactivity) at EVERY point in the sequence.

While we're required to predict these 5 values, only three contribute to the score:<br>
1) **Reactivity**: reactivty values for the first 68 bases as denoted in sequence -- used to determine the likely structure of the RNA sample <br>

2) **deg_Mg_pH10**:used to determine the likelihood of degredation at the base/linkage in question, after incubating with megnesium in high pH (pH10)

3) **deg_Mg_50c**:used to determine the likelihood of degredation at the base/linkage in question, after incubating with megnesium at a high temperature (50 degrees celcius)
--> that's 120+ degrees in fahrenheit, why is this being used as a metric when the competition mentions a concern about degredation during shipment, shouldn't we be concerned with a lower temperature point that could be more representative? 

In [ ]:
#we're getting alot of information in addition to this though
list(train_df.columns)

In [ ]:
train_df.shape, test_df.shape, sample_sub.shape

In [ ]:
#way lower than length of submission file bc it features rows that
#are not present and are meant to be ignored (idk why included)
3634*107 #num test samples * num of basepairs

In [ ]:
#number of positions used in scoring -- 
train_df['seq_scored'].unique()

In [ ]:
#length of sequence -- same for all test samples
train_df['seq_length'].unique()

In [ ]:
#all of the sequences have 107 basepairs (characters)
len(train_df['sequence'][0]), train_df['sequence'][0]

In [ ]:
#distribution of nucleotides in first training sample
collections.Counter(train_df['sequence'][0])

In [ ]:
train_df['structure'][0]

The structure is interesting, it describes whether a base is estimated to be paired or unpaired. Paired bases are denoted by opening and closing parenthesis <br>
(....)<br>
This means that base 0 is paired with base 5 and bases 1-4 are unpaired

What the heck does that mean??

In [ ]:
sample_sequence = train_df.iloc[0]['sequence']
sample_structure = train_df.iloc[0]['structure']
sample_sequence, sample_structure

Copy and paste those two lines into the following: http://rna.tbi.univie.ac.at/forna/ <br>

What you'll see is a visualization of this RNA sequences secondary structure(!) 

In [ ]:
sample_sequence[98:], sample_structure[70:80]

The samples within this sample mRNA that were selected in previous cell are to show that the visual represntation maps onto what's shown in the data. The last few base pairs around 100 seem to match up and the region between 70 and 80 in the structure shows a bunch of base pairs being paired with one another -- which is confirmed by the snippet

So in the training data we are given reactivity and degredation values for the first 68 base pairs and then we are asked to predict values for 107 base pairs. Does it make sense that the first 68 base pairs will be able to provide that much useful insight into the nearly 40 base pairs afterwards? Couldn't it be the case that there's some crazy folding going on later in the sequence? Could you argue that the first 68 base pairs represent a scaffolding for the rest of the molecule? Ehhh, how does the direction of this get determined and is it the same always? 

In [ ]:
#the number of positions used in scoring with predicted values
test_df['seq_scored'].value_counts()

That's the test set -- the reason there is a difference is because in the private test set, you are predicting on 91 positions vs 68 in the public test and training sets

There's more information like *_error_* and degredation values in different conditions, but these are not given in the test data, so we will not consider them right now.

There's also a whole folder "bpps" with numpy arrays representing matrices of base pair probilities... (https://www.kaggle.com/c/stanford-covid-vaccine/discussion/182021) -- some people have been representing them visually, which is a cool idea -- we could feed this visual representation of the data into a model(!) But that's for another time

In [ ]:
test_df.columns

The predicted loop type is a 107 character string that describes the structural context of each character in the sequence.<br>
(S): Paired Stem <br>
(M): Multiloop <br>
(I): Internal Loop <br>
(B): Bulge <br>
(H): Hairpin Loop<br>
(E): Dangling End <br>
(X): external loop

In [ ]:
train_df['predicted_loop_type'][0]

In [ ]:
plt.title('Loop types for single sequence of mRNA')
plt.hist(np.array(list(train_df['predicted_loop_type'][0])));

There's a big range of values in degredation across the conditions

In [ ]:
sample = train_df.iloc[0]
sample

In [ ]:
(np.array(sample['deg_Mg_50C']) > np.array(sample['deg_50C'])).sum() / len(sample['deg_50C'])

Under a third of the base pairs in this sequence have a higher degredation value in the 50 degrees celcius AND magnesium condition vs the 50 degree celcius condition -- does that make sense?

https://pubmed.ncbi.nlm.nih.gov/10488562/ -- uhh??

In [ ]:
(np.array(sample['deg_error_Mg_pH10']) / np.array(sample['deg_error_pH10'])).mean()

The errors in the magnesium condition are 88% bigger than the condition with out it

We'll come back and explore this more in the coming days

In [ ]:
train_df.shape, train_df['reactivity'].shape, len(train_df['reactivity'][0])

So in the training data - we have 2,400 sequence samples - each of these samples has their own reactivity information. Inside of this is a list of values (68) that represent the reactivity of the first 68 base pairs (each get's it's own value).

How are we going  to map this data into a way to feed our models?

The pandas explode fxn seems promising! (https://www.kaggle.com/c/stanford-covid-vaccine/discussion/182118) -- although we need to see it's impact on ordering as mentioned in comments.

Let's try something a bit uglier by hand :)

In [ ]:
#reminder of the columns in our training, test and submission dataframes
list(train_df.columns), list(test_df.columns), list(sample_sub.columns)

Let's reformat the data into the way we need it -- each sequence has X basepairs -- come up with something without looking up an easy solution!

In [ ]:
one_sample = train_df.iloc[0]
sequence = list(one_sample['sequence'])[0:68]
reactivity = one_sample['reactivity']
deg_50C = one_sample['deg_50C']
sample_df = pd.DataFrame([sequence, reactivity, deg_50C]).T
sample_df.columns= ['basepair', 'reactivity', 'deg_50C']
sample_df

We have to relabel each basepair to have a sequence position value which is done like below

In [ ]:
seq_pos = []
for x in range(68):
    seq_pos.append(one_sample['id'] + '_' + str(x))
seq_pos[0:4]

Ok now that we know this works -- let's wrap it up into a fxn

So we want a df with the columns in the test df which represent our independent variables and the columns in our sample submission file which represent our dependent variables

In [ ]:
def extract_sample(sample):
    seq_pos = []
    for x in range(68):
        seq_pos.append(sample['id'] + '_' + str(x))
    sequence = list(sample['sequence'])[0:68]
    structure = list(sample['structure'])[0:68]
    predicted_loop_type = list(sample['predicted_loop_type'])[0:68]
    reactivity = sample['reactivity']
    deg_Mg_pH10 = sample['deg_Mg_pH10']
    deg_pH10 = sample['deg_pH10']
    deg_Mg_50C = sample['deg_Mg_50C']
    deg_50C = sample['deg_50C']
    sample_df = pd.DataFrame([seq_pos, sequence, structure,
                               predicted_loop_type, reactivity,
                              deg_Mg_pH10, deg_pH10, deg_Mg_50C,
                              deg_50C]).T
    sample_df.columns= ['seq_pos', 'basepair', 'structure',
                        'predicted_loop_type', 'reactivity', 
                        'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C',
                        'deg_50C']
                       
    return sample_df

In [ ]:
extract_sample(one_sample)

That was probably more roundabout than needed, but whatever(!)

In [ ]:
%%time
training_data = pd.DataFrame()
for x in range(0, train_df.shape[0]):
    df = extract_sample(train_df.iloc[x])
    training_data = training_data.append(df).reset_index(drop=True)

In [ ]:
training_data.shape

Reformatting the data into a pandas dataframe in this way will make it easier to analyze the data, but we're still missing some columns that will need to be added next time :)\

One minute to do that wasn't terrible -- but if this was a bigger dataset, that way of going about things is probably too hamfisted